In [1]:
# Import dependencies
import os
import anndata as ad
import scanpy as sc
import numpy as np
import pandas as pd

# Print date and time:
import datetime
e = datetime.datetime.now()
print ("Current date and time = %s" % e)

# Set a working directory
wdir = "/mnt/da8aa2c4-0136-465b-87a2-d12a59afec55/akurjan/analysis/files/Training/" #replace with where you want to run the code in
os.chdir( wdir )

Current date and time = 2023-12-23 11:28:43.269477


# Introduction to AnnData

In [2]:
# Create a sample AnnData object
adata = ad.AnnData(
    X=np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), #your counts
    obs={'cell_types': ['A', 'B', 'C']},
    var={'genes': ['Gene1', 'Gene2', 'Gene3']}
)
adata

AnnData object with n_obs × n_vars = 3 × 3
    obs: 'cell_types'
    var: 'genes'

### check the .obs and the .var

In [3]:
adata.obs

,cell_types
0,A
1,B
2,C


In [4]:
adata.var

,genes
0,Gene1
1,Gene2
2,Gene3


### add the below new counts matrix as a layer to your adata object

In [5]:
count_matrix = np.array([[10, 20, 30], [40, 50, 60], [70, 80, 90]])

In [6]:
adata.layers['counts'] = count_matrix.copy()
adata.layers['counts'] 

array([[10, 20, 30],
       [40, 50, 60],
       [70, 80, 90]])

### whenver you change things around, you need to be mindful of how things are created and copied. Run the following code for a reminder of the differences between a deep copy vs a direct assignment:

In [7]:
# Illustrate the difference
# Using .copy()
adata_copy = adata.copy()
adata_copy.X = adata.layers['counts'].copy() #deep copy

# Using direct assignment
adata_direct = adata.copy()
adata_direct.X = adata.layers['counts'] #direct assignment

print("Original data:")
print(adata.layers['counts'])
print("\nData after using .copy():")
print(adata_copy.X)
print("\nData after using direct assignment:")
print(adata_direct.X)

Original data:
[[10 20 30]
 [40 50 60]
 [70 80 90]]

Data after using .copy():
[[10 20 30]
 [40 50 60]
 [70 80 90]]

Data after using direct assignment:
[[10 20 30]
 [40 50 60]
 [70 80 90]]


In [8]:
adata_copy.X[0, 0] = 100  # now we modify the counts in adata_copy.X

# Print the original data to observe changes
print("Original data:")
print(adata.layers['counts'])
print("\nData after using .copy():")
print(adata_copy.X)
print("\nData after using direct assignment:")
print(adata_direct.X)

Original data:
[[10 20 30]
 [40 50 60]
 [70 80 90]]

Data after using .copy():
[[100  20  30]
 [ 40  50  60]
 [ 70  80  90]]

Data after using direct assignment:
[[10 20 30]
 [40 50 60]
 [70 80 90]]


In [9]:
adata_direct.X[0, 0] = 100  # now we modify the counts in adata_direct.X

# Print the original data to observe changes
print("Original data:")
print(adata.layers['counts'])
print("\nData after using .copy():")
print(adata_copy.X)
print("\nData after using direct assignment:")
print(adata_direct.X)

Original data:
[[100  20  30]
 [ 40  50  60]
 [ 70  80  90]]

Data after using .copy():
[[100  20  30]
 [ 40  50  60]
 [ 70  80  90]]

Data after using direct assignment:
[[100  20  30]
 [ 40  50  60]
 [ 70  80  90]]


### Which modification changed the original counts? What does that mean for you going forwards?

Be very careful doing direct assignments and always use .copy() if in doubt

# Exercises

### Reading in .h5 files and converting to anndata

Files stored online can be loaded using the backup_url option like so:

```
adata = sc.read_10x_h5(
    filename="filtered_feature_bc_matrix.h5",
    backup_url="https://figshare.com/ndownloader/files/39546196",
)
```

We are going to work with files provided on the github.

### check that you are in the right folder with your CellRanger data

In [10]:
ls

CellRanger_DEV15984_Ach_Sep2022/   FCAImmFolder1/  QC/
CellRanger_DEV16127_Quad_Jan2023/  FCAImmFolder2/  WSSTraining1/
CellRanger_DEV16569_Ach_Jan2023/   metadata.csv    WSSTraining2/


### 1) load any one single sample object using sc.read_10x_h5() function with a "cellbenderout_filtered.h5" file. Check the adata.obs and adata.var.

In [11]:
adata = sc.read_10x_h5('CellRanger_DEV16127_Quad_Jan2023/cellbenderout_filtered.h5')
adata

/home/akurjan/conda/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 13754 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [12]:
adata.obs

""
CATCCGTTCTATCGTT-1
AGTTCCCTCTCATAGG-1
CCAATTTAGACTTCAC-1
TTCCAATCAGGCATTT-1
ATGACCACACCCTCTA-1
...
AGCGTCGCACCAGCCA-1
TATCTTGAGCTGCGAA-1
GGTGTTATCGTAGGGA-1
CTCAACCTCATCTATC-1


In [13]:
adata.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


### 2) What warning was printed when you loaded the file? Why does it exist? To answer, print out the number of genes in each anndata object, then print out the number of unique genes.

_hint: to find unique genes, you can use the .unique() function._

In [15]:
print(adata.n_vars)
print(len(adata.var_names.unique()))

36601
36591


Answer: there can be multiple Ensembl IDs associated with a single HGNC (Human Gene Nomenclature Committee) symbol. In other words, a single gene symbol (HGNC) can correspond to multiple Ensembl IDs, reflecting alternative transcripts or gene isoforms.

### 3) To fix the warning, run the code below. What does it do? Run your number of genes and number of unique genes again to check what changed.

In [16]:
adata.var_names_make_unique()

In [17]:
print(adata.n_vars)
print(len(adata.var_names.unique()))

36601
36601


### delete this anndata object to free up memory in preparation for next steps

In [18]:
del adata

# Working with multiple anndata objects

### 4) Write a for loop to iterate over all relevant (CellRanger) sample folders in your working directory and load all 'cellbenderout_filtered.h5' files into an adata_dict dictionary object, where sample names are keys and anndata objects are values.

_hint: you'll need to start with a for loop that will have an if condition._

In [ ]:
# solution 1 specific to CellRanger
adata_dict = {}
for folder in os.listdir(wdir):
    if folder.startswith('CellRanger'):
        h5_filepath = os.path.join(folder, 'cellbenderout_filtered.h5')
        adata_dict[folder] = sc.read_10x_h5(h5_filepath)
adata_dict

In [19]:
# solution 2 that loads all cellbenderout_filtered.h5 files that exist in your working directory
adata_dict = {}
for folder in os.listdir(wdir):
    h5_filepath = os.path.join(folder, 'cellbenderout_filtered.h5')
    if os.path.exists(h5_filepath):
        adata_dict[folder] = sc.read_10x_h5(h5_filepath) #sc.read_10x_h5(h5_filepath)
adata_dict

/home/akurjan/conda/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/akurjan/conda/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/akurjan/conda/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


{'CellRanger_DEV16569_Ach_Jan2023': AnnData object with n_obs × n_vars = 10887 × 36601
     var: 'gene_ids', 'feature_types', 'genome',
 'CellRanger_DEV15984_Ach_Sep2022': AnnData object with n_obs × n_vars = 6173 × 36601
     var: 'gene_ids', 'feature_types', 'genome',
 'CellRanger_DEV16127_Quad_Jan2023': AnnData object with n_obs × n_vars = 13754 × 36601
     var: 'gene_ids', 'feature_types', 'genome'}

### 5) Print the number of cells and genes for each of the anndata objects in your adata_dict dictionary. Use a for loop.
_hint: you can use adata.n_obs to get the number of cells and adata.n_vars to get the number of genes_.

In [21]:
for name, anndata in adata_dict.items():
    print(f"{name}: {anndata.n_obs}, {anndata.n_vars}")

CellRanger_DEV16136_Ach_Jan2023: 636, 36601
CellRanger_DEV16569_Ach_Jan2023: 10887, 36601
CellRanger_DEV15984_Ach_Sep2022: 6173, 36601


### 6) Create a list of sample names called 'dict_sample_names' using keys of the 'adata_dict' object.

In [20]:
dict_sample_names = list(adata_dict.keys())
dict_sample_names

['CellRanger_DEV16569_Ach_Jan2023',
 'CellRanger_DEV15984_Ach_Sep2022',
 'CellRanger_DEV16127_Quad_Jan2023']

### 7) Show the .var of the first anndata object within adata_dict using this list. 

In [21]:
adata_dict[dict_sample_names[0]].var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


### 8) Now, let's add some metadata to your anndata. First, load the metadata.csv file into a dataframe called 'df' with the first column as the index of the dataframe.

Metadata was made using the following code:

```
# Provided data
data = {
    'sample_name': ['CellRanger-DEV16569-Ach-Jan2023', 'CellRanger-DEV15984-Ach-Sep2022', 'CellRanger_DEV16127_Quad_Jan2023'],
    'sex': ['female', 'male', 'male'],
    'age': [20, 40, 60]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print("DataFrame:")
print(df)

# Save as a CSV file
csv_filename = 'metadata.csv'
df.to_csv(csv_filename, index=False)

print(f"\nDataFrame saved as {csv_filename}")
```

In [23]:
df = pd.read_csv('metadata.csv', index_col=0)
df

,sex,age
sample_name,,
CellRanger-DEV16569-Ach-Jan2023,female,20
CellRanger-DEV15984-Ach-Sep2022,male,40
CellRanger_DEV16127_Quad_Jan2023,male,60


### 9) We will need to add the df 'sex' and 'age' information to our anndata objects in the 'adata_dict' dictionary. First, check if the sample names in the 'df' dataframe match the names in 'adata_dict' and print out the matching names.

_hint: there are different ways of doing this. One simple way involves writing a for-if loop with df and adata_dict._

In [24]:
# Check if sample names in df match keys in adata_dict
matching_samples = []
for sample_name in df.index:
    if sample_name in adata_dict:
        matching_samples.append(sample_name)
        
# Display the matching sample names
print("Matching sample names:")
print(matching_samples)

Matching sample names:
['CellRanger_DEV16127_Quad_Jan2023']


In [25]:
# A more comprehensive way to write the same thing:
matching_samples = [sample_name for sample_name in df.index if sample_name in adata_dict]
 
# Display the matching sample names
print("Matching sample names:")
print(matching_samples)

Matching sample names:
['CellRanger_DEV16127_Quad_Jan2023']


### 10) If the names do not match, edit them so that they do (DO NOT alter the adata_dict!). Re-run the matching check code from 9.

In [26]:
# Rename the index in the DataFrame by replacing '-' with '_'
df.index = df.index.str.replace('-', '_')
df

,sex,age
sample_name,,
CellRanger_DEV16569_Ach_Jan2023,female,20
CellRanger_DEV15984_Ach_Sep2022,male,40
CellRanger_DEV16127_Quad_Jan2023,male,60


In [27]:
# A more comprehensive way to write the same thing:
matching_samples = [sample_name for sample_name in df.index if sample_name in adata_dict]
 
# Display the matching sample names
print("Matching sample names:")
print(matching_samples)

Matching sample names:
['CellRanger_DEV16569_Ach_Jan2023', 'CellRanger_DEV15984_Ach_Sep2022', 'CellRanger_DEV16127_Quad_Jan2023']


### 11) Add the 'sex' and 'age' variables to your anndata observations in adata_dict by matching to their sample name in df.index. Would they need to be added to the .obs or .var? Check that this has been done correctly by accesing each sample using 'dict_sample_names' like before.
_hint: you will need to use an if statement within a for loop to iterate over sample names in both the index of your dataframe and the keys of adata_dict. Use the df.loc[] indexer within a loop to access and assign values from the DataFrame (df) to the AnnData objects based on the sample names. This allows you to precisely locate and match the 'sex' and 'age' values for each sample in your AnnData objects._

In [ ]:
# if you were working with a single sample and your df had only a single value, your code would look like this:
adata.obs['sex'] = df['sex']
adata.obs['age'] = df['age']

# for multiple samples and different values of 'sex' and 'age', 
# you need to make sure they are matched to the correct sample name, 
# which in this case is stored in the df.index. For this, you can use the df.loc[] indexer functionality:
df.loc[row_label, column_label]

In [34]:
for sample_name, adata in adata_dict.items():
    if sample_name in df.index:
        # Add 'sex' and 'age' as observation variables to the AnnData object
        adata.obs['sex'] = df.loc[sample_name, 'sex']
        adata.obs['age'] = df.loc[sample_name, 'age']

# Display the modified AnnData objects
for sample_name, adata in adata_dict.items():
    print(f"\nSample: {sample_name}")
    print(adata)


Sample: CellRanger_DEV16569_Ach_Jan2023
AnnData object with n_obs × n_vars = 10887 × 36601
    obs: 'sex', 'age'
    var: 'gene_ids', 'feature_types', 'genome'

Sample: CellRanger_DEV15984_Ach_Sep2022
AnnData object with n_obs × n_vars = 6173 × 36601
    obs: 'sex', 'age'
    var: 'gene_ids', 'feature_types', 'genome'

Sample: CellRanger_DEV16127_Quad_Jan2023
AnnData object with n_obs × n_vars = 13754 × 36601
    obs: 'sex', 'age'
    var: 'gene_ids', 'feature_types', 'genome'


In [35]:
adata_dict[dict_sample_names[0]].obs

,sex,age
CACTAAGTCATCCTAT-1,female,20
GCCATTCTCAGGCGAA-1,female,20
CTCAGAAAGAATTGTG-1,female,20
TAACTTCCATTCATCT-1,female,20
TGGCGTGTCCATTTCA-1,female,20
...,...,...
CCGGTGAAGTCGAATA-1,female,20
ATTACCTCAAGTCCCG-1,female,20
TGGTTAGGTTGACGGA-1,female,20
TTCTCTCAGACGTCCC-1,female,20


### 12) Let's create more metadata columns, but now within the anndata objects directly. Our anndata objects currently do not have a 'sample_name' column in their observations. Let's add that in using the adata_dict keys.

In [36]:
for sample_name, adata in adata_dict.items():
    adata.obs['sample_name'] = sample_name
adata_dict[dict_sample_names[0]].obs

,sex,age,sample_name
CACTAAGTCATCCTAT-1,female,20,CellRanger_DEV16569_Ach_Jan2023
GCCATTCTCAGGCGAA-1,female,20,CellRanger_DEV16569_Ach_Jan2023
CTCAGAAAGAATTGTG-1,female,20,CellRanger_DEV16569_Ach_Jan2023
TAACTTCCATTCATCT-1,female,20,CellRanger_DEV16569_Ach_Jan2023
TGGCGTGTCCATTTCA-1,female,20,CellRanger_DEV16569_Ach_Jan2023
...,...,...,...
CCGGTGAAGTCGAATA-1,female,20,CellRanger_DEV16569_Ach_Jan2023
ATTACCTCAAGTCCCG-1,female,20,CellRanger_DEV16569_Ach_Jan2023
TGGTTAGGTTGACGGA-1,female,20,CellRanger_DEV16569_Ach_Jan2023
TTCTCTCAGACGTCCC-1,female,20,CellRanger_DEV16569_Ach_Jan2023


### 13) You'll notice that the sample names follow a consistent format like 'CellRanger_patientID_tissueType_libraryPreparationDate'. Using this sample name, create the 'donor', 'type' and 'libbatch' variables directly in each of your anndata objects.

_hint: you can use the split() function to split the sample name string by a given separator value. Then simply do e.g. adata.obs['donor'] = second part of string_

In [37]:
# Iterate over the items in adata_dict and extract variables from sample names
for sample_name, adata in adata_dict.items():
    # Split the sample name into components using underscores
    components = sample_name.split('_')

    # Extract donor, type, and libbatch from the components
    donor = components[1]  # Assuming patient ID is the second component
    tissue_type = components[2]
    libbatch = components[3]

    # Add 'donor', 'type', and 'libbatch' to observations
    adata.obs['donor'] = donor
    adata.obs['type'] = tissue_type
    adata.obs['libbatch'] = libbatch

In [38]:
adata_dict[dict_sample_names[0]].obs

,sex,age,sample_name,donor,type,libbatch
CACTAAGTCATCCTAT-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
GCCATTCTCAGGCGAA-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
CTCAGAAAGAATTGTG-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
TAACTTCCATTCATCT-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
TGGCGTGTCCATTTCA-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
...,...,...,...,...,...,...
CCGGTGAAGTCGAATA-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
ATTACCTCAAGTCCCG-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
TGGTTAGGTTGACGGA-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023
TTCTCTCAGACGTCCC-1,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023


### 14) Now, using a similar approach, edit the adata.obs indices of all adata_dict anndata objects so that they contain the barcode without the '-1' at the end, the donor name, and the tissue type in the following format: 'Barcode.DonorName_TissueType'.

_hint: barcodes are stored in adata.obs_names and you should make a new 'barcode' obs column to store them in before you alter the index._

In [39]:
for sample_name, adata in adata_dict.items():
    samplename = sample_name.split('_')[1] +'_'+ sample_name.split('_')[2]
    barcode = adata.obs_names.str[:-2] #or you can do replace("-1", "") or adata.obs_names.str.split('-').str[0]
    adata.obs['barcode'] = barcode
    donor = adata.obs['donor'][0]
    tissuetype = adata.obs['type'][0]
    adata.obs.index = barcode + '.' + donor + '_' + tissuetype

In [40]:
adata_dict[dict_sample_names[0]].obs

,sex,age,sample_name,donor,type,libbatch,barcode
CACTAAGTCATCCTAT.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,CACTAAGTCATCCTAT
GCCATTCTCAGGCGAA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,GCCATTCTCAGGCGAA
CTCAGAAAGAATTGTG.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,CTCAGAAAGAATTGTG
TAACTTCCATTCATCT.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TAACTTCCATTCATCT
TGGCGTGTCCATTTCA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TGGCGTGTCCATTTCA
...,...,...,...,...,...,...,...
CCGGTGAAGTCGAATA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,CCGGTGAAGTCGAATA
ATTACCTCAAGTCCCG.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,ATTACCTCAAGTCCCG
TGGTTAGGTTGACGGA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TGGTTAGGTTGACGGA
TTCTCTCAGACGTCCC.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TTCTCTCAGACGTCCC


### 15) Now let's look at anndata.var. First, print out the number of genes in each anndata object, then print out the number of unique genes as before. Make the vars unique if needed.

In [41]:
for adata in adata_dict.values():
    print(adata.n_vars)
    print(len(adata.var_names.unique()))

36601
36591
36601
36591
36601
36591


In [42]:
for adata in adata_dict.values():
    adata.var_names_make_unique()

### 16) Save the anndata objects in the dictionary as .h5ad files in their respective folders. We will use them again tomorrow.

In [ ]:
# to save a single anndata object, you usually run the following command:
adata.write('nameofsample.h5ad')
# to save all the anndata objects from a dictionary, simply write a double for loop using os.listdir() and adata_dict.items().
# use the f-string functionality to automate sample naming, e.g. f'{sample_name}.h5ad'

In [44]:
for folder in os.listdir(wdir):
    for sample_name, adata in adata_dict.items():
        adata.write(os.path.join(sample_name, f'{sample_name}.h5ad'))

### 17) Concatenate the anndata objects together in your adata_dict dictionary using anndata.concat(). 
_hint: If you get an InvalidIndexError, run adata.var_names_make_unique(). you will need to put the anndata objects into a list for anndata.concat() to work. You should also specify join='outer'_

In [45]:
concatenated_adata = ad.concat(list(adata_dict.values()), index_unique=None, join='outer', axis=0)
concatenated_adata

AnnData object with n_obs × n_vars = 30814 × 36601
    obs: 'sex', 'age', 'sample_name', 'donor', 'type', 'libbatch', 'barcode'

In [46]:
concatenated_adata.obs

,sex,age,sample_name,donor,type,libbatch,barcode
CACTAAGTCATCCTAT.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,CACTAAGTCATCCTAT
GCCATTCTCAGGCGAA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,GCCATTCTCAGGCGAA
CTCAGAAAGAATTGTG.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,CTCAGAAAGAATTGTG
TAACTTCCATTCATCT.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TAACTTCCATTCATCT
TGGCGTGTCCATTTCA.DEV16569_Ach,female,20,CellRanger_DEV16569_Ach_Jan2023,DEV16569,Ach,Jan2023,TGGCGTGTCCATTTCA
...,...,...,...,...,...,...,...
AGCGTCGCACCAGCCA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,AGCGTCGCACCAGCCA
TATCTTGAGCTGCGAA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,TATCTTGAGCTGCGAA
GGTGTTATCGTAGGGA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,GGTGTTATCGTAGGGA
CTCAACCTCATCTATC.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,CTCAACCTCATCTATC


### Print out the number of cells for each donor.

In [47]:
concatenated_adata.obs['donor'].value_counts()

donor
DEV16127    13754
DEV16569    10887
DEV15984     6173
Name: count, dtype: int64

In [51]:
concatenated_adata.obs[['type', 'sex', 'donor']].value_counts()

type  sex     donor   
Quad  male    DEV16127    13754
Ach   female  DEV16569    10887
      male    DEV15984     6173
Name: count, dtype: int64

### 18) Create a new anndata object called 'subset' by subset the concatenated object to only male donors.
_hint: since you are subsetting by an obs variable that works as a dataframe, think about how you would filter rows in a dataframe based on a condition. _

In [ ]:
# something like this:
subset = df[df[column] == value to subset to].copy()

In [52]:
subset = concatenated_adata[concatenated_adata.obs['sex'] == 'male'].copy()
subset

AnnData object with n_obs × n_vars = 19927 × 36601
    obs: 'sex', 'age', 'sample_name', 'donor', 'type', 'libbatch', 'barcode'

In [53]:
subset.obs

,sex,age,sample_name,donor,type,libbatch,barcode
AGGTTACTCTATCGTT.DEV15984_Ach,male,40,CellRanger_DEV15984_Ach_Sep2022,DEV15984,Ach,Sep2022,AGGTTACTCTATCGTT
TCTGGCTCACGTACAT.DEV15984_Ach,male,40,CellRanger_DEV15984_Ach_Sep2022,DEV15984,Ach,Sep2022,TCTGGCTCACGTACAT
CCCTCAAGTCATAACC.DEV15984_Ach,male,40,CellRanger_DEV15984_Ach_Sep2022,DEV15984,Ach,Sep2022,CCCTCAAGTCATAACC
AATGACCCAACCTATG.DEV15984_Ach,male,40,CellRanger_DEV15984_Ach_Sep2022,DEV15984,Ach,Sep2022,AATGACCCAACCTATG
GGTAATCCACTGGCCA.DEV15984_Ach,male,40,CellRanger_DEV15984_Ach_Sep2022,DEV15984,Ach,Sep2022,GGTAATCCACTGGCCA
...,...,...,...,...,...,...,...
AGCGTCGCACCAGCCA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,AGCGTCGCACCAGCCA
TATCTTGAGCTGCGAA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,TATCTTGAGCTGCGAA
GGTGTTATCGTAGGGA.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,GGTGTTATCGTAGGGA
CTCAACCTCATCTATC.DEV16127_Quad,male,60,CellRanger_DEV16127_Quad_Jan2023,DEV16127,Quad,Jan2023,CTCAACCTCATCTATC


###  * Non-10X .h5 files might require a more customised function for loading. Here is an example of one that should be applicable to any type of .h5 file

In [ ]:
import tables
import scipy.sparse as sp
import anndata
from typing import Dict, Optional


def anndata_from_h5(file: str,
                    analyzed_barcodes_only: bool = True) -> 'anndata.AnnData':
    """Load an output h5 file into an AnnData object for downstream work.

    Args:
        file: The h5 file
        analyzed_barcodes_only: False to load all barcodes, so that the size of
            the AnnData object will match the size of the input raw count matrix.
            True to load a limited set of barcodes: only those analyzed by the
            algorithm. This allows relevant latent variables to be loaded
            properly into adata.obs and adata.obsm, rather than adata.uns.

    Returns:
        adata: The anndata object, populated with inferred latent variables
            and metadata.

    """

    d = dict_from_h5(file)
    X = sp.csc_matrix((d.pop('data'), d.pop('indices'), d.pop('indptr')),
                      shape=d.pop('shape')).transpose().tocsr()

    # check and see if we have barcode index annotations, and if the file is filtered
    barcode_key = [k for k in d.keys() if (('barcode' in k) and ('ind' in k))]
    if len(barcode_key) > 0:
        max_barcode_ind = d[barcode_key[0]].max()
        filtered_file = (max_barcode_ind >= X.shape[0])
    else:
        filtered_file = True

    if analyzed_barcodes_only:
        if filtered_file:
            # filtered file being read, so we don't need to subset
            print('Assuming we are loading a "filtered" file that contains only cells.')
            pass
        elif 'barcode_indices_for_latents' in d.keys():
            X = X[d['barcode_indices_for_latents'], :]
            d['barcodes'] = d['barcodes'][d['barcode_indices_for_latents']]
        elif 'barcodes_analyzed_inds' in d.keys():
            X = X[d['barcodes_analyzed_inds'], :]
            d['barcodes'] = d['barcodes'][d['barcodes_analyzed_inds']]
        else:
            print('Warning: analyzed_barcodes_only=True, but the key '
                  '"barcodes_analyzed_inds" or "barcode_indices_for_latents" '
                  'is missing from the h5 file. '
                  'Will output all barcodes, and proceed as if '
                  'analyzed_barcodes_only=False')

    # Construct the anndata object.
    adata = anndata.AnnData(X=X,
                            obs={'barcode': d.pop('barcodes').astype(str)},
                            var={'gene_name': (d.pop('gene_names') if 'gene_names' in d.keys()
                                               else d.pop('name')).astype(str)},
                            dtype=X.dtype)
    adata.obs.set_index('barcode', inplace=True)
    adata.var.set_index('gene_name', inplace=True)

    # For CellRanger v2 legacy format, "gene_ids" was called "genes"... rename this
    if 'genes' in d.keys():
        d['id'] = d.pop('genes')

    # For purely aesthetic purposes, rename "id" to "gene_id"
    if 'id' in d.keys():
        d['gene_id'] = d.pop('id')

    # If genomes are empty, try to guess them based on gene_id
    if 'genome' in d.keys():
        if np.array([s.decode() == '' for s in d['genome']]).all():
            if '_' in d['gene_id'][0].decode():
                print('Genome field blank, so attempting to guess genomes based on gene_id prefixes')
                d['genome'] = np.array([s.decode().split('_')[0] for s in d['gene_id']], dtype=str)

    # Add other information to the anndata object in the appropriate slot.
    _fill_adata_slots_automatically(adata, d)

    # Add a special additional field to .var if it exists.
    if 'features_analyzed_inds' in adata.uns.keys():
        adata.var['cellbender_analyzed'] = [True if (i in adata.uns['features_analyzed_inds'])
                                            else False for i in range(adata.shape[1])]

    if analyzed_barcodes_only:
        for col in adata.obs.columns[adata.obs.columns.str.startswith('barcodes_analyzed')
                                     | adata.obs.columns.str.startswith('barcode_indices')]:
            try:
                del adata.obs[col]
            except Exception:
                pass
    else:
        # Add a special additional field to .obs if all barcodes are included.
        if 'barcodes_analyzed_inds' in adata.uns.keys():
            adata.obs['cellbender_analyzed'] = [True if (i in adata.uns['barcodes_analyzed_inds'])
                                                else False for i in range(adata.shape[0])]

    return adata

def dict_from_h5(file: str) -> Dict[str, np.ndarray]:
    """Read in everything from an h5 file and put into a dictionary."""
    d = {}
    with tables.open_file(file) as f:
        # read in everything
        for array in f.walk_nodes("/", "Array"):
            d[array.name] = array.read()
    return d


def _fill_adata_slots_automatically(adata, d):
    """Add other information to the adata object in the appropriate slot."""

    for key, value in d.items():
        try:
            if value is None:
                continue
            value = np.asarray(value)
            if len(value.shape) == 0:
                adata.uns[key] = value
            elif value.shape[0] == adata.shape[0]:
                if (len(value.shape) < 2) or (value.shape[1] < 2):
                    adata.obs[key] = value
                else:
                    adata.obsm[key] = value
            elif value.shape[0] == adata.shape[1]:
                if value.dtype.name.startswith('bytes'):
                    adata.var[key] = value.astype(str)
                else:
                    adata.var[key] = value
            else:
                adata.uns[key] = value
        except Exception:
            print('Unable to load data into AnnData: ', key, value, type(value))